In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
#import xgboost as xgb

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectFromModel
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

We load the training set and the validation set which have been already preprocessed

In [2]:
#Importing dataset
train_complete = pd.read_csv("../dataset/processed1/train_no_val2.csv", index_col=0)
train = train_complete.loc[(train_complete['Date'] <  '2017-11-01')]

val1 = train_complete.loc[(train_complete['Date'] >= '2017-11-0')]
val2 = pd.read_csv("../dataset/processed1/val2.csv", index_col=0)

In [3]:
train.OrdinalDate = train.OrdinalDate.map(lambda x: x % 365)
train_complete.OrdinalDate = train_complete.OrdinalDate.map(lambda x: x % 365)

val1.OrdinalDate = val1.OrdinalDate.map(lambda x: x % 365)
val2.OrdinalDate = val2.OrdinalDate.map(lambda x: x % 365)

D:\Programmi\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


We prepare the training input samples and the labels for all the sets

In [4]:
X=train.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)
X_complete=train_complete.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)
X_val1=val1.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)
X_val2=val2.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)

y=train['NumberOfSales']
y_complete=train_complete['NumberOfSales']
y_val1=val1['NumberOfSales']
y_val2=val2['NumberOfSales']

We try to find good hyperparameter in xGBoost. First of all we try to find good values for max_depth and colsample_bytree 

In [ ]:
depth_range = range(15,17)
col_range = [0.59, 0.6, 0.61]

for d in depth_range:
    for col in col_range:
            xclas = XGBRegressor(subsample=0.7, eta=0.3, max_depth=d, 
                                 colsample_bytree=col, nthread=4,
                                 min_child_weight=5, gamma=0,
                                 n_estimators=50)  
            xclas.fit(X, y)   
            y_pred = xclas.predict(X_val1) 
            mae_val1=mean_absolute_error(y_val1,y_pred)

            print("max_depth= ", d, " colsample_bytree= ", col, " mae= ", mae_val1)

At the end predict the validation error on val2 and we compute the csv in which there is the result

In [5]:
xclas = XGBRegressor(subsample=0.6, eta=0.3, max_depth=15, 
                         colsample_bytree=0.6, nthread=4,
                         min_child_weight=5, gamma=0,
                         n_estimators=50)  

xclas.fit(X_complete, y_complete)  

y_pred_val2 = xclas.predict(X_val2) 
mae_val2=mean_absolute_error(y_val2,y_pred_val2)

print("mae_val2=",mae_val2)

mae_val2= 566.361252644


In [8]:
val2=val2.reset_index(drop=True)
y_val2 = y_val2.reset_index(drop=True)

columns=['Date', 'StoreID', 'RegionID', 'SalesPredicted', 'SalesReal']
index=range(y_val2.shape[0])
result=pd.DataFrame(index=index,columns=columns)

result['Date']=val2['Date']
result['StoreID']=val2['StoreID']
result['RegionID']=val2['Region']
result['SalesPredicted']=y_pred_val2
result['SalesReal']=y_val2

result.to_csv("results_xg.csv")

In [6]:
def errore(ypred, y, val):
    val=val.reset_index(drop=True)
    y = y.reset_index(drop=True)

    columns=['Date', 'StoreID', 'RegionID', 'SalesPredicted', 'SalesReal']
    index=range(y.shape[0])
    result=pd.DataFrame(index=index,columns=columns)

    result['Date']=val['Date']
    result['StoreID']=val['StoreID']
    result['RegionID']=val['Region']
    result['SalesPredicted']=ypred
    result['SalesReal']=y
    

    # Transform dates from '%Y-%m-%d' to datetime objects.
    def transform_date(x):
        date = datetime.datetime.strptime(x, '%Y-%m-%d')
        return date

    result['Date'] = result['Date'].map(transform_date)
    result['Month'] = result.Date.map(lambda d: d.strftime('%Y-%m'))
    result = result.groupby(['StoreID', 'RegionID', 'Month']).sum().reset_index()[['Month', 'StoreID', 'RegionID', 'SalesPredicted', 'SalesReal']]

    result['SalesError'] = abs(result.SalesPredicted - result.SalesReal)

    region_error = (result.groupby('RegionID').sum().SalesError / result.groupby('RegionID').sum().SalesReal).reset_index().rename(columns={0: 'RegionError'}).set_index('RegionID')
    total_error = np.mean(region_error.RegionError)
    return total_error

In [7]:
errore(y_pred_val2, y_val2, val2)

0.06101349388459094